# Animating the Execution of Minimized Finite State Machines

This notebook generates and displays an animation of the steps in a minimized FSM. After executing each cell, you will be able to view the animation at the end of the notebook.

Import `gvanim`, which was installed using `pip install GraphvizAnim`. This module is used to generate the interactive animations. 

In [1]:
from gvanim import Animation
from gvanim.jupyter import interactive
ga_min = Animation() # variable used to represent the animation 

In the section below, we introduce all the classes and methods needed to produce a Minimized Finite State Machine. These methods were referenced from the lecture notes (02 Regular Languages).

In [2]:
class FiniteStateMachine:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(self.F) + '\n' + \
               '\n'.join(r[0] + ' ' + r[1] + ' → ' + r[2] for r in self.R)

def parseFSM(fsm: str) -> FiniteStateMachine:
    fsm = [line for line in fsm.split('\n') if line.strip() != '']
    q0 = fsm[0].split()[0] # first line: initialstate
    F = set(fsm[1].split()) # second line: finalstate, finalstate, ...
    R = set()
    for line in fsm[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateMachine(T, Q, R, q0, F)

class Choice:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Conc:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Star:
    def __init__(self, e): self.e = e
        
def syntaxgraph(re):
    global node, T
    if re == '': return {(None, None)}
    elif type(re) == str:
        node += 1; T.add(re); return {(None, (re, str(node))), ((re, str(node)), None)}
    elif type(re) == Choice:
        return syntaxgraph(re.e1) | syntaxgraph(re.e2)
    elif type(re) == Conc:
        g1, g2 = syntaxgraph(re.e1), syntaxgraph(re.e2)
        return {(a, b) for (a, b) in g1 if b} | \
               {(a, b) for (a, b) in g2 if a} | \
               {(a, b) for (a, c) in g1 for (d, b) in g2 if not c and not d}
    elif type(re) == Star:
        g = syntaxgraph(re.e)
        return {(None, None)} | g | \
               {(a, b) for (a, c) in g for (d, b) in g if not c and not d}
    else: raise Exception('not a regular expression')
        
def convertRegExToFSM(re):
    global node, T; node, T = 0, set()
    g = syntaxgraph(re)
    Q = {str(n) for n in range(node + 1)}
    R = {('0', b[0], b[1]) for (a, b) in g if not a and b} | \
        {(a[1], b[0], b[1]) for (a, b) in g if a and b}
    F = {a[1] for (a, b) in g if a and not b} | ({'0'} if (None, None) in g else set())
    return FiniteStateMachine(T, Q, R, '0', F)

def string(s: set) -> str:
    return '{' + ', '.join(e for e in s) + '}'

def deterministicFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    qq0 = string({fsm.q0})
    QQ, RR, visited = {qq0}, set(), set()
    #print(QQ, RR, visited)
    while visited != QQ:
        qq = (QQ - visited).pop(); visited |= {qq}
        for t in fsm.T:
            rr = {r for (q, u, r) in fsm.R if u == t and q in qq}
            if rr != set(): QQ |= {string(rr)}; RR |= {(qq, t, string(rr))}
        #print(QQ, RR, visited)
    FF = {qq for qq in QQ for f in fsm.F if f in qq}
    return FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)

def renameFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    m, c = {}, 0
    for q in fsm.Q:
        m[q] = str(c); c = c + 1
    QQ = {str(i) for i in range(c)}
    RR = {(m[q], u, m[r]) for (q, u, r) in fsm.R}
    FF = {m[q] for q in fsm.F}
    qq0 = m[fsm.q0]
    output = FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)
    output = str(output)
    return output

def minimizeFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    nxt = {(q, a): r for (q, a, r) in fsm.R}
    dist = {(q, r) for q in fsm.Q for r in fsm.Q if q != r and (q in fsm.F) != (r in fsm.F)}
    done = False #; print('initially ', dist)
    while not done:
        done = True
        for q in fsm.Q:
            for r in fsm.Q:
                if q != r and (q, r) not in dist and any(((q, u) in nxt) != ((r, u) in nxt) or \
                    ((q, u) in nxt) and ((nxt[(q, u)], nxt[(r, u)]) in dist) for u in fsm.T):
                    dist |= {(q, r)}; done = False #; print('adding ', q, r)
        #print('updated ', dist)
    # construct minimal FSM with frozensets as states
    QQ = {frozenset({q} | {r for r in fsm.Q if (q, r) not in dist}) for q in fsm.Q}
    RR = {(qq, u, rr) for qq in QQ for rr in QQ for u in fsm.T if any((q, u, r) in fsm.R for q in qq for r in rr)}
    qq0 = {qq for qq in QQ if any(q in fsm.q0 for q in qq)}.pop()
    FF = {qq for qq in QQ if (qq & fsm.F) != set()}
    # convert frozensets into strings
    QQ = {string(qq) for qq in QQ}
    RR = {(string(qq), u, string(rr)) for (qq, u, rr) in RR}
    qq0 = string(qq0)
    FF = {string(qq) for qq in FF}
    return FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)


###### Defining the Nondeterministic FSM 

First, we need to define the NFA using the methods defined above. We do this by defining a regular expression, and convert that regex into an FSM using `convertRegExToFSM()`. 

In [3]:
E4 = Choice(Conc(Conc('a', Star('a')), 'b'), Conc(Conc('a', Star('a')), 'c'))
A4 = convertRegExToFSM(E4); A4

0
3 6
2 b → 3
4 a → 5
0 a → 4
5 a → 5
2 a → 2
5 c → 6
1 a → 2
1 b → 3
4 c → 6
0 a → 1

###### Defining the Deterministic FSM 

Next, we need to define the equivalent DFA for the above NFA. This is done by inputting the NFA into the method `deterministicFSM()`.

In [4]:
A4det = deterministicFSM(A4); A4det

{0}
{6} {3}
{2, 5} b → {3}
{2, 5} c → {6}
{2, 5} a → {2, 5}
{1, 4} c → {6}
{0} a → {1, 4}
{1, 4} a → {2, 5}
{1, 4} b → {3}

###### Defining the Minimized FSM 

Finally, we minimize the DFA using the method `minimizeFSM()`. 

In [1]:
A4min = minimizeFSM(A4det); A4min

NameError: name 'minimizeFSM' is not defined

We can further simplify the FSM by renaming the states, using the method `renameFSM()`. </br>
We then split the resulting FSM into a list of strings. 

In [6]:
A4simp = renameFSM(A4min); A4simp = A4simp.splitlines()
for i in A4simp:
    print(i)

2
0
1 b → 0
1 a → 1
1 c → 0
2 a → 1


### Printing the Model:

In this section, we define a method for printing a model of the minimized FSM.<br> 
The method `printMinFSM(A)` takes a list of strings as input and produces a model of the minimized FSM, constructed from a series of edges and nodes.  

In [7]:
def printMinFSM(A):
    event = [] # order of events
    if type(A) == list: # only proceeds if input is correct type  
        for i in range(2,len(A)): 
            if type(A[i]) == str: # only proceeds if input is list of strings 
                old = A[i][0:1] 
                transition = A[i][2:3] 
                new = A[i][A[i].find("→")+2:]  
                ga_min.add_edge(old,new) # extend edge from old to new 
                ga_min.label_edge(old,new,transition) # add label for transition 
                event.append([old,transition,new])
            else: # returns error message if incorrect type  
                raise Exception('error: list of strings expected')
        return (event) 
    else: # returns error message if input type is incorrect
        raise Exception('error: list expected') 
printMinFSM(A4simp)

[['1', 'b', '0'], ['1', 'a', '1'], ['1', 'c', '0'], ['2', 'a', '1']]

### Animating the Model:

In this stage, we define a method for animating the model of the minimized FSM.<br> 
The method `animateMinFSM(A)` takes a list of strings as input, and calls the previous defined method `printMinFSM(A)` using this input. It iterates through the list of lists outputted from `printMinFSM(A)`, highlighting each old state, transition, and new state in the minimized FSM. 

In [8]:
def animateMinFSM(A):
    if type(A) == list: # only proceeds if input is correct type
        event = printMinFSM(A) # calls printNFSM(), and uses the output it generates 
        event.sort(key = lambda event: event[1]) 
        event = (sorted(event)) 
        for i in event:
            if type(i) == list:# only proceeds if input is list of lists
                start = list(A[0].strip("{}")) # list of starting states
                final = A[1].replace('{','').replace('}','').split(' ') # list of final states
                if i[0] in start: # highlight starting node first 
                    first = i 
                    ga_min.highlight_node(i[0]) # highlights old state
                    ga_min.next_step()
                    ga_min.highlight_edge(i[0],i[2]) # highlights edge 
                    ga_min.next_step()
                    ga_min.highlight_node(i[2]) # highlights new state
                    event.remove(i)
                    for i in event:
                        if i[len(i)-1] not in final:  
                            first = i 
                            ga_min.highlight_node(i[0]) 
                            ga_min.next_step()
                            ga_min.highlight_edge(i[0],i[2]) 
                            ga_min.next_step()
                            ga_min.highlight_node(i[2]) 
                            event.remove(i)
                            for i in event: # highlight final node last  
                                ga_min.highlight_node(i[0])
                                ga_min.next_step()
                                ga_min.highlight_edge(i[0],i[2])
                                ga_min.next_step()
                                ga_min.highlight_node(i[2])
                                ga_min.next_step()
                            
            else: # returns error message if incorrect type
                raise Exception('error: list of strings expected')
        return event 
    else: # returns error message if input type is incorrect
        raise Exception('error: list expected') 
animateMinFSM(A4simp)

[['2', 'b', '1'], ['2', 'c', '1']]

### Running the Animation:

Here, we call the method `interactive()`, which was imported with the module `gvanim`. This method generates the interative animation using the previously defined methods, `printMinFSM()` and `animateMinFSM()`.

Move the slider from left to right to view the animation of the minimized FSM. 

Note: In this animation, the node for '1' is used twice, to represent the transitions '2 c → 1' and '2 b → 1'. The edges overlap, so it appears as if only one edge is displayed. 

In [9]:
interactive( ga_min, 600 )

interactive(children=(IntSlider(value=0, description='n', max=10), Output()), _dom_classes=('widget-interact',…